<a href="https://colab.research.google.com/github/lituldust/Machine-Learning-Study/blob/Projects/Predict_Health_Cost_with_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
from sklearn.utils import shuffle

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
print(dataset['region'].unique())

In [ ]:
dataset.loc[(dataset['sex'] == 'male'), 'sex'] = 1
dataset.loc[(dataset['sex'] == 'female'), 'sex'] = 0

dataset.loc[(dataset['smoker'] == 'yes'), 'smoker'] = 1
dataset.loc[(dataset['smoker'] == 'no'), 'smoker'] = 0

In [ ]:
dataset.loc[(dataset['region'] == 'southwest'), 'region'] = 0
dataset.loc[(dataset['region'] == 'southeast'), 'region'] = 1
dataset.loc[(dataset['region'] == 'northwest'), 'region'] = 2
dataset.loc[(dataset['region'] == 'northeast'), 'region'] = 3

In [ ]:
dataset.tail()

In [ ]:
dataset = shuffle(dataset).reset_index(drop=True)

In [ ]:
train_dataset  = dataset[0:int(0.8*dataset.shape[0])]
test_dataset = dataset[int(0.8*dataset.shape[0]):dataset.shape[0] - 1]

In [ ]:
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
train_dataset['age'].astype(float)
train_dataset['children'].astype(float)

In [ ]:
train_dataset.info()

In [ ]:
normalizer = layers.experimental.preprocessing.Normalization()
train_dataset = np.array(train_dataset)

In [ ]:
train_dataset = train_dataset.astype('int64')
test_dataset = test_dataset.astype('int64')

In [ ]:
normalizer.adapt(train_dataset)

In [ ]:
model = keras.Sequential([
    normalizer,
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

In [ ]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mae',
    metrics=['mae', 'mse']
)

In [ ]:
model.build()

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_dataset,
    train_labels,
    epochs=100
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
